## Libraries importeren

In [20]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split

## Data pre-processen

In [22]:
df = pd.read_csv("Data/medquad.csv")
df['question'] = df['question'].astype(str)
df['answer'] = df['answer'].astype(str)

df.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


#### Train- en test data maken

In [23]:
train_data, test_data = train_test_split(df, test_size=0.2)
print(len(train_data))
print(len(test_data))

13129
3283


#### Laad het model en de tokenizer

In [24]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

#### Data tokenizen en omzetten naar hugging face dataset

In [ ]:
def tokenize_data(data):
    tokenizer.pad_token = tokenizer.eos_token

    questions = tokenizer(data['question'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    answers = tokenizer(data['answer'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
##Fix dit nog
train_dataset = Dataset.from_pandas(train_data)
train_dataset = train_dataset.map(tokenize_data, batched=True, remove_columns=["text"])
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

test_dataset = Dataset.from_pandas(test_data)
test_dataset = test_dataset.map(tokenize_data, batched=True, remove_columns=["text"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map: 100%|██████████| 3283/3283 [00:03<00:00, 947.37 examples/s]


## Model trainen

In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    seed=42
)

trainer = Trainer(
    model = model,
    args= training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
    )

trainer.train()

C:\Users\Boris\AppData\Local\Temp\ipykernel_22984\2479856785.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.